In [186]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import yfinance

In [187]:
ticker = "INFY.NS"

def get_historical_stock_price(ticker,period='1d',interval='5m'):
    ticker = yfinance.Ticker(ticker)
    return ticker.history(period=period,interval=interval)

In [188]:
data = get_historical_stock_price(ticker, period = '1y', interval = '1d')

In [189]:
data['candle_body_length'] = (data['Close'] - data['Open']).abs()
data['candle_length'] = data['High'] - data['Low']
data['candle_body_ratio'] = (data['candle_body_length'] * 100 / data['candle_length'])
data['day_change_perc'] = data['candle_body_length'] * 100 / data['Close'].shift(1)
data['candle_color'] = np.where(data['Open'] > data['Close'], "Red", np.where(data['Open'] != data['Close'], "Green", "")) 
data['upper_wick_length'] = data['High'] - data[['Close','Open']].max(axis = 1)
data['lower_wick_length'] = data[['Close','Open']].min(axis = 1) - data['Low']

In [190]:
def marubozu(data):
    return pd.Series(np.where(np.logical_and(data['candle_body_ratio'] > 86.5, data['day_change_perc'] > 1.25), 
                                      data['candle_color'], "")).replace({'Red' : 'Bearish', 'Green' : 'Bullish'}).tolist()

def spinning_top_doji(data):
    wick_conditions = np.logical_and((data['upper_wick_length'] / data['candle_length']).between(.25, .75), (data['lower_wick_length'] / data['candle_length']).between(.25, .75))
    body_condition = np.where(data['candle_body_ratio'] <= 17.5, "Doji", np.where(data['candle_body_ratio'] <= 35, "Spinning Top", None))
    return np.where(np.logical_and(body_condition, wick_conditions), body_condition, "")

In [191]:
data['marubozu'] = marubozu(data)

In [192]:
data['spinning_top'] = spinning_top_doji(data)

In [193]:
# paper umbrella
data[np.logical_and(data['candle_body_ratio'] >= 10, np.logical_and((data['lower_wick_length'] / data['candle_length'] > .65),data['lower_wick_length'] >= (1.88 * data['candle_body_length'])))]

,Open,High,Low,Close,Volume,Dividends,Stock Splits,candle_body_length,candle_length,candle_body_ratio,day_change_perc,candle_color,upper_wick_length,lower_wick_length,marubozu,spinning_top
Date,,,,,,,,,,,,,,,,
2021-11-08,1711.870588,1722.755314,1685.153532,1719.291992,5245903,0.0,0,7.421404,37.601782,19.736842,0.438930,Green,3.463322,26.717055,,
2022-01-13,1885.036628,1892.458032,1846.791635,1876.922607,14277630,0.0,0,8.114020,45.666397,17.768032,0.436760,Red,7.421404,30.130973,,
2022-02-01,1747.592261,1761.346621,1716.521290,1753.479980,5119935,0.0,0,5.887720,44.825330,13.134805,0.342707,Green,7.866640,31.070970,,
2022-06-17,1391.050049,1396.949951,1367.150024,1387.300049,9692047,0.0,0,3.750000,29.799927,12.583924,0.268394,Red,5.899902,20.150024,,
2022-08-02,1546.550049,1547.000000,1528.349976,1543.599976,4374635,0.0,0,2.950073,18.650024,15.818066,0.190198,Red,0.449951,15.250000,,
